# LEET CODE
## Problem-1174: Immediate Food Delivery II
Class: Medium

Source: https://leetcode.com/problems/immediate-food-delivery-ii

## Description
Table: Delivery
| Column Name                 | Type    |
|-----------------------------|---------|
| delivery_id                 | int     |
| customer_id                 | int     |
| order_date                  | date    |
| customer_pref_delivery_date | date    |

- delivery_id is the primary key of this table.
- The table holds information about food delivery to customers that make orders at some date and specify a preferred delivery date (on the same order date or after it).

## To Do
If the customer's preferred delivery date is the same as the order date, then the order is called immediate; otherwise, it is called scheduled.

The first order of a customer is the order with the earliest order date that the customer made. It is guaranteed that a customer has precisely one first order.

Write an SQL query to find the percentage of immediate orders in the first orders of all customers, rounded to 2 decimal places.

The query result format is in the following example.

Example 1:

Input: 
Delivery table:
| delivery_id | customer_id | order_date | customer_pref_delivery_date |
|-------------|-------------|------------|-----------------------------|
| 1           | 1           | 2019-08-01 | 2019-08-02                  |
| 2           | 2           | 2019-08-02 | 2019-08-02                  |
| 3           | 1           | 2019-08-11 | 2019-08-12                  |
| 4           | 3           | 2019-08-24 | 2019-08-24                  |
| 5           | 3           | 2019-08-21 | 2019-08-22                  |
| 6           | 2           | 2019-08-11 | 2019-08-13                  |
| 7           | 4           | 2019-08-09 | 2019-08-09                  |

Output: 
| immediate_percentage |
|----------------------|
| 50.00                |

Explanation: 
- The customer id 1 has a first order with delivery id 1 and it is scheduled.
- The customer id 2 has a first order with delivery id 2 and it is immediate.
- The customer id 3 has a first order with delivery id 5 and it is scheduled.
- The customer id 4 has a first order with delivery id 7 and it is immediate.
- Hence, half the customers have immediate first orders.


## Importing

In [1]:
import pandas as pd
from pandasql import sqldf

## Define Schema

In [2]:
data = {
    'delivery_id': [1, 2, 3, 4, 5, 6, 7],
    'customer_id': [1, 2, 1, 3, 3, 2, 4],
    'order_date': [
        '2019-08-01',
        '2019-08-02',
        '2019-08-11',
        '2019-08-24',
        '2019-08-21',
        '2019-08-11',
        '2019-08-09'
    ],
    'customer_pref_delivery_date': [
        '2019-08-02',
        '2019-08-02',
        '2019-08-12',
        '2019-08-24',
        '2019-08-22',
        '2019-08-13',
        '2019-08-09'
    ]
}

Delivery = pd.DataFrame(data)
Delivery['order_date'] = pd.to_datetime(Delivery['order_date'])
Delivery['customer_pref_delivery_date'] = pd.to_datetime(Delivery['customer_pref_delivery_date'])

Delivery

,delivery_id,customer_id,order_date,customer_pref_delivery_date
0,1,1,2019-08-01,2019-08-02
1,2,2,2019-08-02,2019-08-02
2,3,1,2019-08-11,2019-08-12
3,4,3,2019-08-24,2019-08-24
4,5,3,2019-08-21,2019-08-22
5,6,2,2019-08-11,2019-08-13
6,7,4,2019-08-09,2019-08-09


## Task

In [28]:
# Define the SQL query
query = """
SELECT ROUND(100 * SUM(CASE WHEN order_date=customer_pref_delivery_date THEN 1 ELSE 0 END )/CAST(COUNT(*) AS float), 2) AS immediate_percentage
FROM (
    SELECT *,
           ROW_NUMBER() OVER (PARTITION BY customer_id ORDER BY order_date) AS rn
    FROM Delivery
) ranked
WHERE rn = 1
;
"""

# Excute the query using pandasql
result = sqldf(query, env={'Delivery':Delivery})

# Display the result dataframe
display(result)

,immediate_percentage
0,50.0


ROW_NUMBER() is a window function in SQL that assigns a unique integer value to each row within a result set based on the specified ordering. It's often used in combination with the OVER() clause to define a partitioning of the result set and the ordering of rows within each partition.

Here's a basic syntax of how ROW_NUMBER() is used:

```sql

ROW_NUMBER() OVER (PARTITION BY partition_expression ORDER BY order_expression)
```

PARTITION BY: This clause divides the result set into partitions. The ROW_NUMBER() function restarts numbering for each partition. If you omit this clause, the function treats the entire result set as a single partition.

ORDER BY: This clause specifies the order in which the rows are assigned the row numbers within each partition. Rows are numbered in ascending order of the specified column(s).